## [요구사항 1] titanic_dataset.py 분석

In [1]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader, random_split

In [2]:
class TitanicDataset(Dataset):
  def __init__(self, X, y):
    self.X = torch.FloatTensor(X)
    self.y = torch.LongTensor(y)

  def __len__(self):
    return len(self.X)

  def __getitem__(self, idx):
    feature = self.X[idx]
    target = self.y[idx]
    return {'input': feature, 'target': target}

  def __str__(self):
    str = "Data Size: {0}, Input Shape: {1}, Target Shape: {2}".format(
      len(self.X), self.X.shape, self.y.shape
    )
    return str

TitanicDataset  
학습 데이터를 위한 DataSet  
feature(입력) target(출력)

In [3]:
class TitanicTestDataset(Dataset):
  def __init__(self, X):
    self.X = torch.FloatTensor(X)

  def __len__(self):
    return len(self.X)

  def __getitem__(self, idx):
    feature = self.X[idx]
    return {'input': feature}

  def __str__(self):
    str = "Data Size: {0}, Input Shape: {1}".format(
      len(self.X), self.X.shape
    )
    return str

TitanicTestDataset  
테스트 데이터를 위한 데이터셋  
feature(입력)

In [4]:
def get_preprocessed_dataset():
    CURRENT_FILE_PATH = os.path.abspath('')

    train_data_path = os.path.join(CURRENT_FILE_PATH, "train.csv")
    test_data_path = os.path.join(CURRENT_FILE_PATH, "test.csv")

    train_df = pd.read_csv(train_data_path)
    test_df = pd.read_csv(test_data_path)

    all_df = pd.concat([train_df, test_df], sort=False)

    all_df = get_preprocessed_dataset_1(all_df)

    all_df = get_preprocessed_dataset_2(all_df)

    all_df = get_preprocessed_dataset_3(all_df)

    all_df = get_preprocessed_dataset_4(all_df)

    all_df = get_preprocessed_dataset_5(all_df)

    all_df = get_preprocessed_dataset_6(all_df)

    train_X = all_df[~all_df["Survived"].isnull()].drop("Survived", axis=1).reset_index(drop=True)
    train_y = train_df["Survived"]

    test_X = all_df[all_df["Survived"].isnull()].drop("Survived", axis=1).reset_index(drop=True)

    dataset = TitanicDataset(train_X.values, train_y.values)
    #print(dataset)
    train_dataset, validation_dataset = random_split(dataset, [0.8, 0.2])
    test_dataset = TitanicTestDataset(test_X.values)
    #print(test_dataset)

    return train_dataset, validation_dataset, test_dataset

get_preprocessed_dataset()  
CSV 파일을 읽어와 전처리를 수행한 뒤,  
train, validation, test 데이터셋을 생성 및 반환한다.

In [5]:
def get_preprocessed_dataset_1(all_df):
    # Pclass별 Fare 평균값을 사용하여 Fare 결측치 메우기
    Fare_mean = all_df[["Pclass", "Fare"]].groupby("Pclass").mean().reset_index()
    Fare_mean.columns = ["Pclass", "Fare_mean"]
    all_df = pd.merge(all_df, Fare_mean, on="Pclass", how="left")
    all_df.loc[(all_df["Fare"].isnull()), "Fare"] = all_df["Fare_mean"]

    return all_df


def get_preprocessed_dataset_2(all_df):
    # name을 세 개의 컬럼으로 분리하여 다시 all_df에 합침
    name_df = all_df["Name"].str.split("[,.]", n=2, expand=True)
    name_df.columns = ["family_name", "honorific", "name"]
    name_df["family_name"] = name_df["family_name"].str.strip()
    name_df["honorific"] = name_df["honorific"].str.strip()
    name_df["name"] = name_df["name"].str.strip()
    all_df = pd.concat([all_df, name_df], axis=1)

    return all_df

결측치(Missing Value): 값이 없는 데이터 

preprocessed_1: 평균값으로 결측치 채우기
1. Pclass와 Fare를 Pclass로 그룹화 후 평균값을 계산한다.
2. all_df와 Fare_mean을 Pclass를 기준으로 병합한다.
3. Fare의 결측치를 Fare_mean 값으로 채운다.

preprocessed_2: name을 family_name, honorific, name 컬럼으로 분리
1. name을 ',' 구분자를 기준으로 세개로 분리한 뒤,
2. 각각 family_name, honorific, name 컬럼으로 저장한다.
3. 컬럼의 양쪽 공백을 제거한다.
4. all_df와 결합한다.

In [6]:
def get_preprocessed_dataset_3(all_df):
    # honorific별 Age 평균값을 사용하여 Age 결측치 메우기
    honorific_age_mean = all_df[["honorific", "Age"]].groupby("honorific").median().round().reset_index()
    honorific_age_mean.columns = ["honorific", "honorific_age_mean", ]
    all_df = pd.merge(all_df, honorific_age_mean, on="honorific", how="left")
    all_df.loc[(all_df["Age"].isnull()), "Age"] = all_df["honorific_age_mean"]
    all_df = all_df.drop(["honorific_age_mean"], axis=1)

    return all_df


def get_preprocessed_dataset_4(all_df):
    # 가족수(family_num) 컬럼 새롭게 추가
    all_df["family_num"] = all_df["Parch"] + all_df["SibSp"]

    # 혼자탑승(alone) 컬럼 새롭게 추가
    all_df.loc[all_df["family_num"] == 0, "alone"] = 1
    all_df["alone"].fillna(0, inplace=True)

    # 학습에 불필요한 컬럼 제거
    all_df = all_df.drop(["PassengerId", "Name", "family_name", "name", "Ticket", "Cabin"], axis=1)

    return all_df

preprocessed_3: Age 평균값으로 결측치 채우기
1. honorific과 Age를 honorific으로 그룹화 후 Age의 평균값을 계산한다.
2. all_df와 honorific_age_mean을 honorific을 기준으로 병합한다.
3. Age의 결측치를 honorific_age_mean 값으로 채운다.

preprocessed_4: 가족수, 혼자탑승 컬럼을 추가하고 학습에 불필요한 컬럼을 제거한다.

In [7]:
def get_preprocessed_dataset_5(all_df):
    # honorific 값 개수 줄이기
    all_df.loc[
    ~(
            (all_df["honorific"] == "Mr") |
            (all_df["honorific"] == "Miss") |
            (all_df["honorific"] == "Mrs") |
            (all_df["honorific"] == "Master")
    ),
    "honorific"
    ] = "other"
    all_df["Embarked"].fillna("missing", inplace=True)

    return all_df


def get_preprocessed_dataset_6(all_df):
    # 카테고리 변수를 LabelEncoder를 사용하여 수치값으로 변경하기
    category_features = all_df.columns[all_df.dtypes == "object"]
    from sklearn.preprocessing import LabelEncoder
    for category_feature in category_features:
        le = LabelEncoder()
        if all_df[category_feature].dtypes == "object":
          le = le.fit(all_df[category_feature])
          all_df[category_feature] = le.transform(all_df[category_feature])

    return all_df

preprocessed_5: honorific 컬럼 축소 및 Embarked 결측치 채우기
1. honorific 컬럼에 Mr, Miss, Mrs, Master 이외의 값을 missing으로 채운다.

preprocessed_6: 카테고리 변수를 수치값으로 인코딩한다.

In [8]:
from torch import nn
class MyModel(nn.Module):
  def __init__(self, n_input, n_output):
    super().__init__()

    self.model = nn.Sequential(
      nn.Linear(n_input, 30),
      nn.ReLU(),
      nn.Linear(30, 30),
      nn.ReLU(),
      nn.Linear(30, n_output),
    )

  def forward(self, x):
    x = self.model(x)
    return x

MyModel: 신경망 모델  
nn.Module을 상속하여 사용자 모델 정의  
nn.Sequential을 사용하여 모델의 구조 정의: 레이어와 활성화함수  
foward 함수 정의: 모델 통과

In [10]:
def test(test_data_loader):
  print("[TEST]")
  batch = next(iter(test_data_loader))
  print("{0}".format(batch['input'].shape))
  my_model = MyModel(n_input=11, n_output=2)
  output_batch = my_model(batch['input'])
  prediction_batch = torch.argmax(output_batch, dim=1)
  for idx, prediction in enumerate(prediction_batch, start=892):
      print(idx, prediction.item())

MyModel의 모델평가에 사용한다.  
테스트 데이터를 모델에 통과시키고 output batch에서 큰 값을 선택하여 출력한다.

In [9]:
if __name__ == "__main__":
  train_dataset, validation_dataset, test_dataset = get_preprocessed_dataset()

  print("train_dataset: {0}, validation_dataset.shape: {1}, test_dataset: {2}".format(
    len(train_dataset), len(validation_dataset), len(test_dataset)
  ))
  print("#" * 50, 1)

  for idx, sample in enumerate(train_dataset):
    print("{0} - {1}: {2}".format(idx, sample['input'], sample['target']))

  print("#" * 50, 2)

  train_data_loader = DataLoader(dataset=train_dataset, batch_size=16, shuffle=True)
  validation_data_loader = DataLoader(dataset=validation_dataset, batch_size=16, shuffle=True)
  test_data_loader = DataLoader(dataset=test_dataset, batch_size=len(test_dataset))

  print("[TRAIN]")
  for idx, batch in enumerate(train_data_loader):
    print("{0} - {1}: {2}".format(idx, batch['input'].shape, batch['target'].shape))

  print("[VALIDATION]")
  for idx, batch in enumerate(validation_data_loader):
    print("{0} - {1}: {2}".format(idx, batch['input'].shape, batch['target'].shape))

  print("#" * 50, 3)

  test(test_data_loader)

train_dataset: 713, validation_dataset.shape: 178, test_dataset: 418
################################################## 1
0 - tensor([ 3.0000,  1.0000, 29.0000,  0.0000,  0.0000,  7.8958,  2.0000, 13.3029,
         2.0000,  0.0000,  1.0000]): 0
1 - tensor([ 1.0000,  0.0000, 22.0000,  0.0000,  1.0000, 55.0000,  2.0000, 87.5090,
         1.0000,  1.0000,  0.0000]): 1
2 - tensor([ 3.0000,  1.0000, 29.0000,  0.0000,  0.0000,  7.7333,  1.0000, 13.3029,
         2.0000,  0.0000,  1.0000]): 0
3 - tensor([ 3.0000,  0.0000, 22.0000,  0.0000,  0.0000,  8.0500,  2.0000, 13.3029,
         1.0000,  0.0000,  1.0000]): 0
4 - tensor([ 3.0000,  1.0000, 29.0000,  0.0000,  0.0000,  7.2292,  0.0000, 13.3029,
         2.0000,  0.0000,  1.0000]): 0
5 - tensor([ 1.0000,  1.0000, 50.0000,  1.0000,  0.0000, 55.9000,  2.0000, 87.5090,
         2.0000,  1.0000,  0.0000]): 0
6 - tensor([ 3.0000,  1.0000, 33.0000,  0.0000,  0.0000,  7.8958,  2.0000, 13.3029,
         2.0000,  0.0000,  1.0000]): 0
7 - tensor([ 3.00

NameError: name 'test' is not defined

주요 실행  
1. 전처리 및 dataset 생성
2. 각 dataset 크기 출력
3. train_dataset의 입출력값을 출력
4. 각 dataset으로 DataLoader 생성 및 출력
5. 모델평가 수행

## [요구사항 2] 딥러닝 모델 훈련 코드 및 Activation Function 변경해보기

In [26]:
from torch import nn
class MyModel(nn.Module):
    def __init__(self, n_input, n_output):
        super().__init__()

        self.model = nn.Sequential(
            nn.Linear(n_input, 30),
            nn.ReLU(),
            nn.Linear(30, 30),
            nn.ReLU(),
            nn.Linear(30, n_output),
        )

    def forward(self, x):
        x = self.model(x)
        return x


def test(test_data_loader, model):
    print("[TEST]")
    batch = next(iter(test_data_loader))
    print("{0}".format(batch['input'].shape))
    output_batch = model(batch['input'])
    prediction_batch = torch.argmax(output_batch, dim=1)

    submission = []
    for idx, prediction in enumerate(prediction_batch, start=892):
        print(idx, prediction.item())

In [12]:
import torch
from torch import nn, optim
from torch.utils.data import random_split, DataLoader
from datetime import datetime
import wandb
import argparse

import os
from pathlib import Path
BASE_PATH = str(Path(os.path.abspath('')).resolve().parent.parent.parent) # BASE_PATH: /Users/yhhan/git/link_dl

import sys
sys.path.append(BASE_PATH)

In [13]:
def get_data():
  train_dataset, validation_dataset, test_dataset = get_preprocessed_dataset()

  train_data_loader = DataLoader(dataset=train_dataset, batch_size=wandb.config.batch_size, shuffle=True)
  validation_data_loader = DataLoader(dataset=validation_dataset, batch_size=wandb.config.batch_size, shuffle=True)
  test_data_loader = DataLoader(dataset=test_dataset, batch_size=len(test_dataset))

  return train_data_loader, validation_data_loader, test_data_loader

  def forward(self, x):
    x = self.model(x)
    return x

In [32]:
def get_model_and_optimizer():
  my_model = MyModel(n_input=11, n_output=2) # input, output 변경
  optimizer = optim.SGD(my_model.parameters(), lr=wandb.config.learning_rate)

  return my_model, optimizer

In [38]:
def training_loop(model, optimizer, train_data_loader, validation_data_loader):
  n_epochs = wandb.config.epochs
  loss_fn = nn.CrossEntropyLoss()  # Loss함수 변경(CrossEntropy): target은 이진 분류
  next_print_epoch = 100

  for epoch in range(1, n_epochs + 1):
    loss_train = 0.0
    num_trains = 0
    for train_batch in train_data_loader:
      output_train = model(train_batch['input'])
      loss = loss_fn(output_train, train_batch['target'])
      loss_train += loss.item()
      num_trains += 1

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

    loss_validation = 0.0
    num_validations = 0
    with torch.no_grad():
      for validation_batch in validation_data_loader:
        output_validation = model(validation_batch['input'])
        loss = loss_fn(output_validation, validation_batch['target'])
        loss_validation += loss.item()
        num_validations += 1

    wandb.log({
      "Epoch": epoch,
      "Training loss": loss_train / num_trains,
      "Validation loss": loss_validation / num_validations
    })

    if epoch >= next_print_epoch:
      print(
        f"Epoch {epoch}, "
        f"Training loss {loss_train / num_trains:.4f}, "
        f"Validation loss {loss_validation / num_validations:.4f}"
      )
      next_print_epoch += 100

In [33]:
def main(args):
  current_time_str = datetime.now().astimezone().strftime('%Y-%m-%d_%H-%M-%S')

  config = {
    'epochs': args.epochs,
    'batch_size': args.batch_size,
    'learning_rate': 1e-3,
    'n_hidden_unit_list': [30, 30],
  }

  wandb.init(
    mode="online" if args.wandb else "disabled",
    project="titanic_model_training",
    notes="My first wandb experiment",
    tags=["my_model", "titanic"],
    name=current_time_str,
    config=config
  )
  print(args)
  print(wandb.config)

  train_data_loader, validation_data_loader, test_data_loader = get_data()

  linear_model, optimizer = get_model_and_optimizer()

  wandb.watch(linear_model)

  print("#" * 50, 1)

  training_loop(
    model=linear_model,
    optimizer=optimizer,
    train_data_loader=train_data_loader,
    validation_data_loader=validation_data_loader
  )
  wandb.finish()

  test(test_data_loader, linear_model) #테스트 수행

In [34]:
# https://docs.wandb.ai/guides/track/config
if __name__ == "__main__":
    ignore_options = ['-f'] # jupyter notebook exception handling

    parser = argparse.ArgumentParser()

    parser.add_argument("--wandb", action=argparse.BooleanOptionalAction, default=True, help="True or False")
    parser.add_argument("-b", "--batch_size", type=int, default=16, help="Batch size (int, default: 16)")
    parser.add_argument("-e", "--epochs", type=int, default=1_000, help="Number of training epochs (int, default: 1,000)")

    sys.argv = [arg for arg in sys.argv if arg not in ignore_options]

    args = parser.parse_args()

    main(args)

Namespace(wandb=True, batch_size=16, epochs=1000)
{'epochs': 1000, 'batch_size': 16, 'learning_rate': 0.001, 'n_hidden_unit_list': [30, 30]}
################################################## 1
Epoch 100, Training loss 0.5257, Validation loss 0.5213
Epoch 200, Training loss 0.4739, Validation loss 0.4958
Epoch 300, Training loss 0.4466, Validation loss 0.4225
Epoch 400, Training loss 0.4351, Validation loss 0.4194
Epoch 500, Training loss 0.4321, Validation loss 0.4252
Epoch 600, Training loss 0.4166, Validation loss 0.4421
Epoch 700, Training loss 0.4257, Validation loss 0.4780
Epoch 800, Training loss 0.4187, Validation loss 0.3824
Epoch 900, Training loss 0.4172, Validation loss 0.3779
Epoch 1000, Training loss 0.4067, Validation loss 0.3981


Epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
Training loss,█▆▆▅▅▄▄▄▃▃▃▃▂▂▃▂▂▂▂▂▂▂▂▂▁▂▂▂▁▁▂▁▂▁▁▂▁▁▂▁
Validation loss,▇█▇▅▆▇▆▄▄▅▄▂▆▄▂▄▅▃▃▂▄▇▄▃▁▂▁▁▁▂▂▃▇▆▃▃▁▂▃▂
Epoch,1000
Training loss,0.4067
Validation loss,0.39807


[TEST]
torch.Size([418, 11])
892 0
893 0
894 0
895 0
896 1
897 0
898 1
899 0
900 1
901 0
902 0
903 0
904 1
905 0
906 1
907 1
908 0
909 0
910 0
911 0
912 0
913 0
914 1
915 1
916 0
917 0
918 1
919 0
920 0
921 0
922 0
923 0
924 0
925 0
926 0
927 0
928 1
929 1
930 0
931 0
932 0
933 0
934 0
935 1
936 1
937 0
938 0
939 0
940 1
941 0
942 0
943 0
944 0
945 0
946 0
947 0
948 0
949 0
950 0
951 0
952 0
953 0
954 0
955 1
956 1
957 1
958 1
959 0
960 0
961 0
962 1
963 0
964 0
965 0
966 0
967 0
968 0
969 1
970 0
971 1
972 0
973 0
974 0
975 0
976 0
977 0
978 1
979 1
980 1
981 1
982 1
983 0
984 1
985 0
986 1
987 0
988 1
989 0
990 1
991 0
992 1
993 0
994 0
995 0
996 1
997 0
998 0
999 0
1000 0
1001 0
1002 0
1003 1
1004 1
1005 1
1006 0
1007 0
1008 0
1009 1
1010 1
1011 1
1012 1
1013 0
1014 1
1015 0
1016 0
1017 1
1018 0
1019 0
1020 0
1021 0
1022 0
1023 0
1024 0
1025 0
1026 0
1027 0
1028 0
1029 0
1030 1
1031 0
1032 0
1033 1
1034 0
1035 0
1036 0
1037 0
1038 0
1039 0
1040 0
1041 0
1042 1
1043 0
1044 0
1045 0
1

#### Wandb 그래프 & Activation Function  

WandbURL: https://wandb.ai/sapigr/titanic_model_training  
![TrainingLoss](https://github.com/leekyrim/PublicRepo/blob/main/trainingLoss.PNG?raw=true)  
![validationLoss](https://github.com/leekyrim/PublicRepo/blob/main/validationLoss.PNG?raw=true)  

**Activation Function**
1. ReLU
2. ELU
3. LeakyReLU
4. PReLU
5. Sigmoid
6. Tanh
7. SELU
8. GELU
  
![Activation Function](https://github.com/leekyrim/PublicRepo/blob/main/activationFunction.PNG?raw=true)

전체적으로 Loss 그래프가 균일하고 수치가 우수한 **SELU**를 선택했습니다.

## [요구사항 3] 테스트 및 submission.csv 생성

In [35]:
from torch import nn
class MyModel(nn.Module):
    def __init__(self, n_input, n_output):
        super().__init__()

        self.model = nn.Sequential(
            nn.Linear(n_input, 30),
            nn.SELU(), #활성함수 변경
            nn.Linear(30, 30),
            nn.SELU(), #활성함수 변경
            nn.Linear(30, n_output),
        )

    def forward(self, x):
        x = self.model(x)
        return x

In [36]:
def test(test_data_loader, model):
    print("[TEST]")
    batch = next(iter(test_data_loader))
    print("{0}".format(batch['input'].shape))
    output_batch = model(batch['input'])
    prediction_batch = torch.argmax(output_batch, dim=1)

    # CSV Export
    submission = []
    for idx, prediction in enumerate(prediction_batch, start=892):
        print(idx, prediction.item())
        row = {"PassengerId": idx, "Survived": prediction.item()}
        submission.append(row)

    submission_df = pd.DataFrame(submission)
    submission_df.to_csv("submission.csv", index=False)

In [37]:
# https://docs.wandb.ai/guides/track/config
if __name__ == "__main__":
    ignore_options = ['-f']

    parser = argparse.ArgumentParser()

    parser.add_argument("--wandb", action=argparse.BooleanOptionalAction, default=True, help="True or False")
    parser.add_argument("-b", "--batch_size", type=int, default=16, help="Batch size (int, default: 16)")
    parser.add_argument("-e", "--epochs", type=int, default=1_000, help="Number of training epochs (int, default: 1,000)")

    sys.argv = [arg for arg in sys.argv if arg not in ignore_options]

    args = parser.parse_args()

    main(args)

Namespace(wandb=True, batch_size=16, epochs=1000)
{'epochs': 1000, 'batch_size': 16, 'learning_rate': 0.001, 'n_hidden_unit_list': [30, 30]}
################################################## 1
Epoch 100, Training loss 0.5473, Validation loss 0.5528
Epoch 200, Training loss 0.5102, Validation loss 0.5075
Epoch 300, Training loss 0.4850, Validation loss 0.5294
Epoch 400, Training loss 0.4698, Validation loss 0.4921
Epoch 500, Training loss 0.4513, Validation loss 0.4411
Epoch 600, Training loss 0.4289, Validation loss 0.4430
Epoch 700, Training loss 0.4430, Validation loss 0.4481
Epoch 800, Training loss 0.4262, Validation loss 0.5661
Epoch 900, Training loss 0.4097, Validation loss 0.6453
Epoch 1000, Training loss 0.4013, Validation loss 0.4765


Epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
Training loss,█▇▇▆▆▆▅▅▅▄▄▃▄▄▃▃▂▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▂▁▁▁▂▁▁▁
Validation loss,▄▄▄▃▄▃▃▃▅▃▂▄▂▂▂▂▂▄▁▂▂▁▁▂▂▂▂▄▂▂▂▁█▂▃▄▁▅▂▁
Epoch,1000
Training loss,0.40128
Validation loss,0.47648


[TEST]
torch.Size([418, 11])
892 0
893 0
894 0
895 0
896 1
897 0
898 1
899 0
900 1
901 0
902 0
903 0
904 1
905 0
906 1
907 1
908 0
909 0
910 1
911 0
912 0
913 0
914 1
915 1
916 1
917 0
918 1
919 0
920 0
921 0
922 0
923 0
924 0
925 0
926 0
927 0
928 1
929 1
930 0
931 0
932 0
933 0
934 0
935 1
936 1
937 0
938 0
939 0
940 1
941 0
942 0
943 0
944 1
945 1
946 0
947 0
948 0
949 0
950 0
951 1
952 0
953 0
954 0
955 1
956 1
957 1
958 1
959 0
960 0
961 1
962 1
963 0
964 1
965 0
966 1
967 1
968 0
969 1
970 0
971 1
972 1
973 1
974 0
975 0
976 0
977 0
978 1
979 1
980 1
981 1
982 1
983 0
984 1
985 0
986 0
987 0
988 1
989 0
990 1
991 0
992 1
993 0
994 0
995 0
996 1
997 0
998 0
999 0
1000 0
1001 0
1002 0
1003 1
1004 1
1005 1
1006 1
1007 0
1008 0
1009 1
1010 0
1011 1
1012 1
1013 0
1014 1
1015 0
1016 0
1017 0
1018 0
1019 0
1020 0
1021 0
1022 0
1023 0
1024 0
1025 0
1026 0
1027 0
1028 0
1029 0
1030 1
1031 0
1032 0
1033 1
1034 1
1035 0
1036 0
1037 0
1038 0
1039 0
1040 0
1041 0
1042 1
1043 0
1044 0
1045 0
1

## [요구사항 4] submission.csv 제출 및 등수확인

![Kaggle](https://github.com/leekyrim/PublicRepo/blob/main/Kaggle.PNG?raw=true)

## [요구사항 5] Wandb 페이지 생성 및 URL 제출

WandbURL: https://wandb.ai/sapigr/titanic_model_training  
![TrainingLoss](https://github.com/leekyrim/PublicRepo/blob/main/trainingLoss.PNG?raw=true)  
![validationLoss](https://github.com/leekyrim/PublicRepo/blob/main/validationLoss.PNG?raw=true)

## 기술적 고찰
데이터 전처리 과정과 데이터 셋을 나누는 과정을 이해했습니다.  
모델 구축 과정에서 여러 활성함수를 시도해봤습니다.  


처음에 model training 코드를 titanic 데이터에 맞춰 수정하며  
output의 dimension이 다르고, output이 왜 두개인지 이해가 안됐습니다.  
코드를 다시 한번 분석해보니 주어진 문제는 0과 1로 출력이 나오는 이진분류 문제임을 깨닫고  
Loss 함수를 MSE에서 CrossEntropy로 변경했습니다.  

또한 Layer, BatchSize, Learning Rate, Epoch 등 파라미터를 조정하며 테스트해봤습니다.  
Loss의 값 그래프를 비교해봤을때, 잘못된 파라미터를 선택하면 Loss 함수가 불균일 하거나 Loss값이 크게 줄지 않았습니다.

## 숙제 후기  
과제를 시작했을때는 간단하게 생각했었으나,  
트레이닝한 모델을 사용하지 않고 테스트를 수행하는 등 엉뚱한 실수를 많이 했습니다.  
샘플 코드를 분석하면서 굉장히 잘 작성된 코드라고 느꼈고 Wandb를 처음 사용해봐서 신기했습니다.  
파라미터 값을 조정하며 어떤 파라미터가 시도해봐야하고 정답일지 궁금했습니다.
